# CNN

In [ ]:
import os, random, time
import numpy as np
import nibabel as nib
import cv2
import matplotlib.pyplot as plt

from utilities import *

# Pytorch functions
import torch
# Neural network layers
import torch.nn as nn
import torch.nn.functional as F
# Optimizer
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
# Torchvision library
from torchvision import transforms

from sklearn.model_selection import train_test_split
# For results
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from torchsummary import summary


In [ ]:
batch_size = 256
N_EPOCHS = 1

In [ ]:
# Device configuration
if torch.cuda.is_available():
    device = torch.device('cuda')
# elif torch.backends.mps.is_available():
#    device = torch.device('mps')
else:
    device = torch.device('cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

In [ ]:
def set_seed(seed, use_cuda = True, use_mps = False):
    """
    Set SEED for PyTorch reproducibility
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_cuda:
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    if use_mps:
        torch.mps.manual_seed(seed)

SEED = 44

USE_SEED = True

if USE_SEED:
    set_seed(SEED, torch.cuda.is_available(), torch.backends.mps.is_available())

# MRI Slice Inputs

In [ ]:
class BraTSDataset(Dataset):
    def __init__(self, image_path = r'./BraTS/BraTS2021_Training_Data', transform=None):
        'Initialisation'
        self.image_path = image_path
        self.folders_name = [folder for folder in os.listdir(self.image_path) if folder != '.DS_Store']
        self.transform = transform

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.folders_name) * 155

    def __getitem__(self, index):
        'Generates one sample of data'

        # Determine the image index and the RGB layer
        image_idx = index // 155
        layer_idx = index % 155

        # Select sample
        fld_name = self.folders_name[image_idx]
        image = []
        for scan_type in ['flair', 't1ce', 't2']:
            path_img = os.path.join(self.image_path, fld_name, fld_name + '_' + scan_type + '.nii.gz')
            img = nib.load(path_img).get_fdata()
            # Need to apply standardisation here...
            image.append(img[:, :, layer_idx])
        
        image = np.array(image)

        path_label = os.path.join(self.image_path, fld_name, fld_name + '_seg.nii.gz')

        label = nib.load(path_label).get_fdata()[:, :, layer_idx]
        
        if self.transform:
            image, label = self.transform([image, label])
        return image, label

In [ ]:
class everythirdlayer(object):
    def __call__(self, sample):
        image, label = sample
        new_image = image[:,:,:,np.arange(3, 152, 3)]
        new_label = label[:,:,np.arange(3, 152, 3)]
        return new_image, new_label

class Flair(object):
    def __call__(self, sample):
        image, label = sample
        new_image = image[0]
        return new_image, label
    
class IntLabel(object):
    def __call__(self, sample):
        image, label = sample
        new_label = label.astype(int)
        return image, new_label
    
class BinariseLabel(object):
    def __call__(self, sample):
        image, label = sample
        new_label = np.sign(label)
        return image, new_label

class CropAndResize(object):
    def __call__(self, sample):
        image, label = sample
        rows = np.any(image, axis=1)
        cols = np.any(image, axis=0)

        # Find the bounding box of the non-zero regions
        rows_indices = np.where(rows)[0]
        cols_indices = np.where(cols)[0]
        if len(rows_indices) != 0 or len(cols_indices) != 0:
            top_row = np.min(rows_indices)
            bottom_row = np.max(rows_indices)
            left_col = np.min(cols_indices)
            right_col = np.max(cols_indices)

            square_size = max(bottom_row - top_row, right_col - left_col) + 1

            # Crop the image
            cropped_image = image[top_row:top_row + square_size, left_col:left_col + square_size]
            cropped_label = label[top_row:top_row + square_size, left_col:left_col + square_size]
        else:
            cropped_image = image
            cropped_label = label
        # Resize the image
        dim = [64,64]
        resized_image = cv2.resize(cropped_image, dim)
        resized_label = cv2.resize(cropped_label, dim)

        return resized_image, resized_label

class Standardise(object) :
    def __call__(self, sample):
        image, label = sample
        if np.abs(image).sum() == 0:
            return image, label
        with np.errstate(divide='ignore',invalid='ignore'):
            image_no_zeros = np.where(image == 0, np.nan, image)
            new_image = (image_no_zeros - np.nanmean(image_no_zeros)) / np.nanstd(image_no_zeros)
            new_image = np.nan_to_num(new_image)
        return new_image, label

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        image, label = sample
        image = np.float32(image)
        label = np.float32(label)

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        # image = image.transpose((2, 0, 1))
        return torch.from_numpy(image), torch.from_numpy(label)

In [ ]:
dataset = BraTSDataset(image_path = r'./BraTS/BraTS2021_Training_Data',
                                    transform=transforms.Compose([
                                        Flair(),
                                        CropAndResize(),
                                        Standardise(),
                                        IntLabel(),
                                        BinariseLabel(),
                                        ToTensor()
                                    ]))

# Post processing scans

In [ ]:
plt.style.use('default')
fig, axes = plt.subplots(4,4, figsize=(12,12))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(dataset[i * 10][0])
plt.show()
fig, axes = plt.subplots(4,4, figsize=(12,12))
for i, ax in enumerate(axes.reshape(-1)):
    ax.imshow(dataset[i * 10][1])
plt.show()

In [ ]:
# train_val_test_split = [0.7, 0.2, 0.1]

# generator = torch.Generator().manual_seed(SEED)

dataset_size = int(len(dataset)/155)
dataset_indices = list(range(dataset_size))

train_indices, test_indices = train_test_split(dataset_indices, test_size=0.1, random_state=SEED)
train_indices, val_indices = train_test_split(train_indices, test_size=0.22, random_state=SEED)

In [ ]:
tmp_list = [[],[],[]]
for i, ind_list in enumerate([train_indices, val_indices, test_indices]):
    for ind in ind_list:
        for j in range(155):
            tmp_list[i].append(ind*155 + j)
train_indices, val_indices, test_indices = tmp_list

In [ ]:
train_subset = Subset(dataset, train_indices)
val_subset = Subset(dataset, val_indices)
test_subset = Subset(dataset, test_indices)

# Create the subset DataLoader
train_dataloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_subset, batch_size=batch_size, shuffle=True)

## PyTorch Docs

[nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d)

[nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

[nn.ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html)

In [ ]:
def Conv2d_output(i, p, d, k, s):
    return (i + 2*p - d*(k-1) - 1)/s + 1
def ConvTranspose2d_output(i, p, d, k, s): # p is output_padding
    return (i-1)*s+d*(k-1)+p+1

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.features = nn.Sequential(
      ## encoder layers ##
      # conv layer (depth from 1 --> 4), 3x3 kernels
      # Input 64 x 64
      nn.Conv2d(in_channels=1, out_channels=4, kernel_size=3, padding = 'same'), # 64 x 64
      nn.ReLU(),
      # pooling layer to reduce x-y dims by two; kernel and stride of 2
      nn.MaxPool2d(2), ## 32 x 32
      # conv layer (depth from 4 --> 8), 4x4 kernels
      nn.Conv2d(in_channels=4, out_channels=8, kernel_size=3, padding = 'same'), # 32 x 32
      nn.ReLU(),
      nn.MaxPool2d(2), # 16 x 16
      # conv layer (depth from 8 --> 12), 5x5 kernels
      nn.Conv2d(in_channels=8, out_channels=12, kernel_size=3, padding = 'same'), # ( 12 x ) 16 x 16
      nn.ReLU(),
      
      ## decoder layers ##
      # add transpose conv layers, with relu activation function
      nn.ConvTranspose2d(12, 6, kernel_size = 2, stride=2), # 32 x 32
      nn.ReLU(),
      nn.ConvTranspose2d(6, 1, kernel_size = 2, stride=2), # 64 x 64
      # output layer (with sigmoid for scaling from 0 to 1)
      # nn.Sigmoid()
    )
    
  def forward(self, x):
    x = x.view(x.shape[0], 1, 64, 64)
    x = self.features(x)
    # x = x.view(x.shape[0], -1)
    # x = torch.flatten(x, start_dim=1)
    return x

In [ ]:
model = CNN().to(device)

print(f"The model has {count_parameters(model):,} trainable parameters.")

In [ ]:
# Loss
# criterion = nn.CrossEntropyLoss() # Softmax + CrossEntropy
# criterion = nn.BCELoss()
criterion = torch.nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

# Optim
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_losses, train_accs, valid_losses, valid_accs = model_training(N_EPOCHS,
                                                                    model,
                                                                    train_dataloader,
                                                                    val_dataloader,
                                                                    optimizer,
                                                                    criterion,
                                                                    device,
                                                                    'CNN.pt')

In [ ]:
model_testing(model, test_dataloader, criterion, device, 'CNN.pt')
print_report(model, test_dataloader, device)